The notebook verifies the binary Hamiltonian recognition algorithm in [1] that matches the claim in Theorem 1. Note that the package `quairkit` is required to run the notebook. The package can be installed by running `pip install quairkit`.

In [1]:
import quairkit as qkit
from quairkit import Circuit
from quairkit.loss import Measure
from quairkit.database import *

import numpy as np
from numpy.polynomial.polynomial import Polynomial
from numpy.polynomial.chebyshev import cheb2poly

from wx_angles import quantum_signal_processing

qkit.set_dtype('complex128')

## Algorithm of Binary Hamiltonian Recognition

**Input**: $k$ copies of an evolution dynamic $U_{H}(\theta) = e^{-iH \theta}$, where $k$ is odd, $t \in \mathbb{R}$ is unknown and $H$ is an unknown Hamiltonian from a known set $\{X, Z\}$.

**Output**: 1 bit as a guess of $H$, where '0' stands for $Z$ and '1' stands for $X$.

### Step 1. Compute Angle

Determine the vector of angles $\vec{\phi} = \phi_0, \ldots, \phi_k$ such that
$$
\bra{0} R_z(\phi_0) \prod_{j=1}^k W_x(a) R_z(\phi_j) \ket{0}  = \frac{2}{k+1} \sum_{l=1,\,l\text{ odd}}^{k} T_l(a)
,$$
The angle computation logic is referred to Theorem 3-5 in [2], and is implemented by the qsvt module in [3].

In [2]:
def phi_set(k: int) -> np.ndarray:
    r"""Compute the angle for the QSP circuit.
    
    Args:
        k: depth
        
    Returns:
        phi angles for Wx-based QSP
    
    """
    chebyshev_coef = np.zeros([k + 1])
    for i in range(k + 1):
        if i % 2 == 1:
            chebyshev_coef[i] = -1
    chebyshev_coef = chebyshev_coef * 2 / (k + 1)
    
    P = Polynomial(cheb2poly(chebyshev_coef))
    return quantum_signal_processing(P) * -2

### Step 2: Construct circuit

Construct the single-qubit circuit
$$
    {\rm QSP}_{H,k}(\theta) = R_z(\phi_0) \prod_{j=1}^k U_{H}(\theta) R_z(\phi_j)
.$$

In [3]:
def _qsp(label: str, list_phi: np.ndarray, theta: np.ndarray) -> Circuit:
    cir = Circuit(1)
    cir.rz(param=list_phi[0])
    
    for phi in list_phi[1:]:
        if label == '0':
            cir.rz(param=theta)
        else:
            cir.rx(param=theta)
        cir.rz(param=phi)
    return cir

### Step 3: Run circuit

Input the zero state $\ket{0}$, run the circuit. Then perform a computational basis measurement at the end, and return the measurement outcome.

In [4]:
num_shot = 1000
M = Measure()

def algorithm1(label: str, k: int) -> float:
    r"""Implementation of the algorithm 1 for 1000 angles uniformly sampled from [0, 2 * pi).
    
    Args:
        label: label of the input unknown evolution operator
        k: number of available copies
    
    Returns:
        the average success probability of correctly guessing the input label
    
    """
    list_theta = np.linspace(0, 2 * np.pi, num_shot)
    
    list_phi = phi_set(k)
    cir = _qsp(label, list_phi, list_theta) # 1000 circuits
    output_state = cir()
    
    success_prob = M(output_state, desired_result=label)
    return success_prob.mean().item()

## Verification

Theorem 1 in [1] states that, the theoretical average success probability of this algorithm is $(2k + 1) / (2k + 2)$. We can show that for odd $k \in \{3, \ldots, 15\}$, the worst experimental error is no larger than $0.001$.

In [5]:
def experiment(k: int) -> int:
    r"""Perform the experiment for a given k
    
    Args:
        k: number of available copies
    
    Returns:
        the experimental average probability of success
    
    """
    assert k % 2 == 1 and 3 <= k <= 15

    success_z, success_x = algorithm1('0', k), algorithm1('1', k)
    return (success_z + success_x) / 2

In [6]:
list_k = np.array([15])
list_ideal_success = ((2 * list_k + 1) / (2 * list_k + 2))

list_experiment_success = np.array([experiment(k) for k in list_k])

In [7]:
print('The maximal gap between the ideal and experimental success probability is', np.max(np.abs(list_ideal_success - list_experiment_success)))

The maximal gap between the ideal and experimental success probability is 0.0004687500000937206


---

## References

[1] C. Zhu, S. He, Y. Chen, L. Zhang, and X. Wang, Optimal Hamiltonian Recognition of Unknown Quantum Dynamics (2024), arXiv:2412.13067.

[2] A. Gilyén, Y. Su, G. H. Low, and N. Wiebe, Quantum singular value transformation and beyond: exponential improvements for quantum matrix arithmetics, in Proceedings of the 51st Annual ACM SIGACT Symposium on Theory of Computing, STOC 19 (ACM, 2019).

[3] Baidu Research Quantum, Paddle Quantum, 2020.

*Table: A reference of notation conventions.*

| Symbol        | Description                               |
|:---------------:|-------------------------------------------|
|  $\phi_j, \theta$ |  rotation angles  |
|  $R_x, R_z$ |   the rotation gates w.r.t. $x, z$ axis |
|  $W_x(a)$ |  $R_x(-2 \arccos(a))$  |
|  $T_l$ |  $l$-th Chebyshev polynomial of the first kind  |


In [8]:
qkit.print_info()


---------VERSION---------
quairkit: 0.3.0
torch: 2.5.1+cpu
numpy: 1.26.0
scipy: 1.14.1
matplotlib: 3.10.0
---------SYSTEM---------
Python version: 3.10.15
OS: Windows
OS version: 10.0.26100
---------DEVICE---------
CPU: ARMv8 (64-bit) Family 8 Model 1 Revision 201, Qualcomm Technologies Inc
